In [2]:
import sqlite3
import pandas as pd
con = sqlite3.connect(r"C:\Users\Mrinalini\Desktop\ML_HW\nfl.db")
df_stats_table = pd.read_sql_query("SELECT * FROM 'stats_team'", con)
# df_stats_table = df_stats_table[df_stats_table.columns[~df_stats_table.columns.isin(['boxscore_id'])]]
df_stats_table.head()

,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,...,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions
0,2002,1,200209050nyg,sfo,0,26,16,166,166.0,1,...,0,0,1.0,27:32,0,0,4,12,0,0
1,2002,1,200209050nyg,nyg,1,45,28,342,318.0,0,...,0,0,3.0,32:28,3,24,9,16,0,1
2,2002,1,200209080mia,det,0,36,17,220,206.0,1,...,2,1,2.0,25:36,3,14,7,14,0,1
3,2002,1,200209080mia,mia,1,27,18,207,207.0,3,...,2,0,0.0,34:24,0,0,6,12,1,1
4,2002,1,200209080jax,clt,0,31,19,211,203.0,3,...,2,2,2.0,27:27,1,8,9,14,0,0


In [3]:
# Below code loads the data from game_info table and since there are half the no. of rows present in the stats_team table......
# we will duplicate every row(using individual lists for each colum) and place the pairs together.....
# logically weather conditions are same for both the teams....
# Finally the result would be appended to the stats_team's dataframe.......
# to get the result after adding the win loss list.....
df_gameData_Weather = pd.read_sql_query("select roof,surface,temp,humidity,wind_chill,wind_mph from game_info", con)
gameData_Roof_List = []
gameData_Surface_List = []
gameData_Temp_List = []
gameData_Humidity_List = []
gameData_Wind_Chill_List = []
gameData_wind_Mph_List = []
for index,row in df_gameData_Weather.iterrows():
    roof,surface,temp,humidity,wind_chill,wind_mph = row['roof'],row['surface'],row['temp'],row['humidity'],row['wind_chill'],row['wind_mph']
    gameData_Roof_List.append(roof)
    gameData_Roof_List.append(roof)
    gameData_Surface_List.append(surface)
    gameData_Surface_List.append(surface)
    gameData_Temp_List.append(temp)
    gameData_Temp_List.append(temp)
    gameData_Humidity_List.append(humidity)
    gameData_Humidity_List.append(humidity)
    gameData_Wind_Chill_List.append(wind_chill)
    gameData_Wind_Chill_List.append(wind_chill)
    gameData_wind_Mph_List.append(wind_mph)
    gameData_wind_Mph_List.append(wind_mph)
    
# df_gameData_Weather modified with the contents of the list
df_gameData_Weather = pd.DataFrame({'roof': gameData_Roof_List, 'surface': gameData_Surface_List, 'temp': gameData_Temp_List, 'humidity': gameData_Humidity_List, 'wind_chill': gameData_Wind_Chill_List, 'wind_mph': gameData_wind_Mph_List})
# Delete the lists as the data has been imported to dataframe
del gameData_Roof_List, gameData_Surface_List, gameData_Temp_List, gameData_Humidity_List, gameData_Wind_Chill_List, gameData_wind_Mph_List

# Append the new dataframe to the stats_team dataframe
df_stats_table['roof'] = df_gameData_Weather['roof'] 
df_stats_table['surface'] = df_gameData_Weather['surface']
df_stats_table['temp'] = df_gameData_Weather['temp']
df_stats_table['humidity'] = df_gameData_Weather['humidity']
df_stats_table['wind_chill'] = df_gameData_Weather['wind_chill']
df_stats_table['wind_mph'] = df_gameData_Weather['wind_mph']
df_stats_table.head()

,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,...,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,roof,surface,temp,humidity,wind_chill,wind_mph
0,2002,1,200209050nyg,sfo,0,26,16,166,166.0,1,...,4,12,0,0,outdoors,grass,73.0,49.0,0.0,7.0
1,2002,1,200209050nyg,nyg,1,45,28,342,318.0,0,...,9,16,0,1,outdoors,grass,73.0,49.0,0.0,7.0
2,2002,1,200209080mia,det,0,36,17,220,206.0,1,...,7,14,0,1,outdoors,grass,87.0,69.0,0.0,12.0
3,2002,1,200209080mia,mia,1,27,18,207,207.0,3,...,6,12,1,1,outdoors,grass,87.0,69.0,0.0,12.0
4,2002,1,200209080jax,clt,0,31,19,211,203.0,3,...,9,14,0,0,outdoors,grass,85.0,0.0,0.0,16.0


In [4]:
# Now work on adding the win loss in binary representation
df_game_table = pd.read_sql_query("select away_score, home_score from game_info", con)
df_game_table.head(5)

,away_score,home_score
0,16.0,13.0
1,21.0,49.0
2,28.0,25.0
3,23.0,27.0
4,7.0,10.0


In [5]:
df_game_table = pd.DataFrame(df_game_table.values).T
df_game_table

,0,1,2,3,4,5,6,7,8,9,...,4529,4530,4531,4532,4533,4534,4535,4536,4537,4538
0,16.0,21.0,28.0,23.0,7.0,40.0,34.0,37.0,34.0,23.0,...,22.0,23.0,16.0,13.0,22.0,28.0,14.0,26.0,37.0,3.0
1,13.0,49.0,25.0,27.0,10.0,39.0,37.0,31.0,6.0,31.0,...,24.0,17.0,15.0,31.0,30.0,41.0,20.0,23.0,31.0,13.0


In [6]:
df_4m_GameInfo = pd.DataFrame()
df_4m_GameInfo = df_game_table[df_game_table.columns[0]]
for i in range(4539):
    df_4m_GameInfo = df_4m_GameInfo.append(df_game_table[df_game_table.columns[i]], ignore_index=True)
df_4m_GameInfo = df_4m_GameInfo.drop([0,1])
df_4m_GameInfo.index = range(len(df_4m_GameInfo))
df_4m_GameInfo.head()

0    16.0
1    13.0
2    21.0
3    49.0
4    28.0
dtype: float64

In [7]:
# Scores is added to a new row in the dataframe pertaining to table stats_team in the provided database
df_stats_table['Score'] = df_4m_GameInfo
df_stats_table['Score'] = df_stats_table['Score'].astype(int)
df_stats_table.head(6)

,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,...,third_down_conversions,fourth_down_attempts,fourth_down_conversions,roof,surface,temp,humidity,wind_chill,wind_mph,Score
0,2002,1,200209050nyg,sfo,0,26,16,166,166.0,1,...,12,0,0,outdoors,grass,73.0,49.0,0.0,7.0,16
1,2002,1,200209050nyg,nyg,1,45,28,342,318.0,0,...,16,0,1,outdoors,grass,73.0,49.0,0.0,7.0,13
2,2002,1,200209080mia,det,0,36,17,220,206.0,1,...,14,0,1,outdoors,grass,87.0,69.0,0.0,12.0,21
3,2002,1,200209080mia,mia,1,27,18,207,207.0,3,...,12,1,1,outdoors,grass,87.0,69.0,0.0,12.0,49
4,2002,1,200209080jax,clt,0,31,19,211,203.0,3,...,14,0,0,outdoors,grass,85.0,0.0,0.0,16.0,28
5,2002,1,200209080jax,jax,1,36,22,228,225.0,2,...,16,3,3,outdoors,grass,85.0,0.0,0.0,16.0,25


In [8]:
# win loss is calculated and stored as binary o,1 in the list which finally will be added to 
# the dataframe pertaining to stats_team after verifying total entries in the list below at the end
win_loss_List = []
for i in range(int(len(df_stats_table)/2)):
    if(df_stats_table['Score'][int(i*2)] > df_stats_table['Score'][int(i*2+1)]):
        win_loss_List.append('1')
        win_loss_List.append('0')
    else:
        win_loss_List.append('0')
        win_loss_List.append('1')
len(win_loss_List)   

9078

In [9]:
# Have a look at the win_loss_list
win_loss_List[:10]

['1', '0', '0', '1', '1', '0', '0', '1', '0', '1']

In [10]:
df_stats_table['Won'] = win_loss_List 
df_stats_table.head()

,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,...,fourth_down_attempts,fourth_down_conversions,roof,surface,temp,humidity,wind_chill,wind_mph,Score,Won
0,2002,1,200209050nyg,sfo,0,26,16,166,166.0,1,...,0,0,outdoors,grass,73.0,49.0,0.0,7.0,16,1
1,2002,1,200209050nyg,nyg,1,45,28,342,318.0,0,...,0,1,outdoors,grass,73.0,49.0,0.0,7.0,13,0
2,2002,1,200209080mia,det,0,36,17,220,206.0,1,...,0,1,outdoors,grass,87.0,69.0,0.0,12.0,21,0
3,2002,1,200209080mia,mia,1,27,18,207,207.0,3,...,1,1,outdoors,grass,87.0,69.0,0.0,12.0,49,1
4,2002,1,200209080jax,clt,0,31,19,211,203.0,3,...,0,0,outdoors,grass,85.0,0.0,0.0,16.0,28,1


In [11]:
df_stats_table.to_csv('NFL_datasetForML.csv')

In [12]:
%pwd

'C:\\Users\\Mrinalini\\Desktop\\ML_HW'